In [2]:
import numpy as np
import pandas as pd

In [126]:
centroids = pd.read_csv("DGL_data/NUTS2_cents_4326.csv", sep=',', encoding="latin-1")
testi_matkat = pd.read_csv('DGL_data/Real_data/OD_long-term_students_2015.csv', sep=',', encoding="utf-8", usecols=['OD_ID','ORIGIN','DESTINATION','COUNT'])
testi_matkat = testi_matkat.drop_duplicates(subset=['OD_ID'])

print(type(testi_matkat))
print(testi_matkat)

<class 'pandas.core.frame.DataFrame'>
          OD_ID ORIGIN DESTINATION  COUNT
0     AL02_AT33   AL02        AT33    1.0
1     AL02_NL21   AL02        NL21    2.0
2     AL02_PL42   AL02        PL42    1.0
3     AL02_UKH1   AL02        UKH1    1.0
4     AL02_UKH3   AL02        UKH3    1.0
...         ...    ...         ...    ...
3594  UKN0_IE06   UKN0        IE06  119.0
3595  UKN0_NL32   UKN0        NL32    1.0
3596  UKN0_PT11   UKN0        PT11    1.0
3597  UKN0_PT15   UKN0        PT15    1.0
3598  UKN0_SI04   UKN0        SI04    1.0

[3599 rows x 4 columns]


In [137]:
testi_matkat_FI = testi_matkat[testi_matkat['ORIGIN'].str.contains('FI')]
testi_matkat_FI = testi_matkat_FI.reset_index(drop=True)
print(type(testi_matkat_FI))
print(testi_matkat_FI)

<class 'pandas.core.frame.DataFrame'>
        OD_ID ORIGIN DESTINATION  COUNT
0   FI19_DE11   FI19        DE11    3.0
1   FI19_DE21   FI19        DE21    2.0
2   FI19_DE25   FI19        DE25    1.0
3   FI19_DE30   FI19        DE30    1.0
4   FI19_DE50   FI19        DE50    1.0
5   FI19_DE71   FI19        DE71    1.0
6   FI19_DE91   FI19        DE91    2.0
7   FI19_DEA2   FI19        DEA2    4.0
8   FI19_DEB2   FI19        DEB2    1.0
9   FI19_FR10   FI19        FR10    1.0
10  FI19_ITI3   FI19        ITI3    1.0
11  FI19_RO11   FI19        RO11    1.0
12  FI1B_DE11   FI1B        DE11    1.0
13  FI1B_DE21   FI1B        DE21    1.0
14  FI1B_DE30   FI1B        DE30    2.0
15  FI1B_DE71   FI1B        DE71    5.0
16  FI1B_DE91   FI1B        DE91    1.0
17  FI1B_DEE0   FI1B        DEE0    1.0
18  FI1B_IS00   FI1B        IS00    1.0
19  FI1B_ITI4   FI1B        ITI4    1.0
20  FI1B_NL41   FI1B        NL41    1.0
21  FI1B_PL91   FI1B        PL91    1.0
22  FI1B_SE11   FI1B        SE11    2.0
23

In [128]:
def split(points, smoothing):
    p = points
    # for each level of smoothing, insert new control point in the middle of all points already in the array
    # loop starts from 1 => smoothing level of 1 keeps only node points
    for smooth in range(1, smoothing):
        new_points = []
        for i in range(len(p) - 1):
            new_point = np.divide(p[i] + p[i + 1], 2.0)
            new_points.append(p[i])
            new_points.append(new_point)
        new_points.append(p[-1])
        p = new_points
    return p

In [143]:
def get(source, dest, nodes, path, smoothing):
    control_points = []
    current_node = source
    for edge_in_path in path:
        control_points.append(np.array([current_node.longitude, current_node.latitude]))

        other_node_id = edge_in_path.destination if edge_in_path.source == current_node.id else edge_in_path.source
        current_node = nodes[other_node_id]
    
    control_points.append(np.array([dest.longitude, dest.latitude]))
    return split(control_points, smoothing)

IDs.PY

In [130]:
def create_ids(dataframe):
        
    # Create a object of unique locations (union of ORIGIN and DESTINATION)
    #unique_locations = dataframe['orig_nuts'].append(dataframe['dest_nuts']).unique()
    unique_locations = dataframe.drop_duplicates(subset=['NUTS_ID'])

    # Create a mapping between unique locations and consecutive integers
    location_mapping = dict(zip(unique_locations['NUTS_ID'], range(1, len(unique_locations) + 1)))

    # Map source and destination ids to consecutive integers
    #dataframe['source_id'] = dataframe['orig_nuts'].map(location_mapping)
    #dataframe['destination_id'] = dataframe['dest_nuts'].map(location_mapping)
    
    return location_mapping

MODEL.PY

In [131]:
import math

class Edge:

    def __init__(self, source, destination, od_id, count):
        self.source = source
        self.destination = destination
        self.od_id = od_id
        self.count = count
        self.distance = -1
        self.weight = -1
        self.skip = False
        self.lock = False


class Node:

    def __init__(self, id, longitude, latitude, name):
        self.id = id
        self.longitude = longitude
        self.latitude = latitude
        self.name = name
        self.edges = []

        # dijkstra related attributes
        self.distance = -1
        self.visited = False
        self.previous = None
        self.previous_edge = None

    def distance_to(self, other) -> float:
        return math.sqrt(pow(other.longitude - self.longitude, 2) + pow(other.latitude - self.latitude, 2))

    def __lt__(self, other):
        return self.id < other.id

    def __gt__(self, other):
        return self.id > other.id

    def __le__(self, other):
        return self.id <= other.id

    def __ge__(self, other):
        return self.id >= other.id

LOCATIONS.PY

In [139]:
def get_locations_data(d):
    # Load data into dataframes
    nodes_list = centroids
    edges_list = testi_matkat_FI

    nodes = {}
    edges = []
    
    #nodes_list['id'] = nodes_list.reset_index().index + 1 
    node_ids = create_ids(nodes_list)

    # Load nodes into dict. Maps ID -> Node instance
    for index, row in nodes_list.iterrows():
        idx = node_ids[row['NUTS_ID']]
        name = row['NUTS_ID']
        lat = row['Y']
        long = row['X']
        nodes[idx] = Node(idx, long, lat, name)

    # Load edges to list
    for index, row in edges_list.iterrows():
        so = node_ids[row['ORIGIN']]
        dest = node_ids[row['DESTINATION']]
        od_id = row['OD_ID']
        count = row['COUNT']
        edges.append(Edge(source=so, destination=dest, od_id=od_id, count=count))
    
    # set iterator for edge removal
    calc_r = 0

    # Assign edges to nodes
    for edge in edges:

        # eliminate edges without nodes
        if edge.destination not in nodes or edge.source not in nodes:
            calc_r += 1
            edges.remove(edge)

        source = nodes[edge.source]
        dest = nodes[edge.destination]
        distance = source.distance_to(dest)

        edge.distance = distance
        edge.weight = pow(distance, d)

        source.edges.append(edge)
        dest.edges.append(edge)
    
    # print removed edges
    print('Total number of edges removed: ' + str(calc_r))

    # iterator for removed nodes
    calc_n = 0
    
    # Eliminate nodes without edges
    to_remove = [node.id for node in nodes.values() if len(node.edges) == 0]
    for key in to_remove:
        calc_n += 1
        del nodes[key]
    
    # print removed node count
    print('Nodes removed: '+str(calc_n))
    
    # print nodes values
    print(nodes.values())

    # Sort edges inside nodes in ascending order
    for node in nodes.values():
        node.edges.sort(key=lambda x: x.distance)

    # Sort edges
    edges.sort(key=lambda x: x.weight, reverse=True)
    print(edges[0].source)
    print(edges[0].destination)
    print(edges[0].od_id)
    print('Origin node name: ' + str(nodes[edges[0].source].name) + ' , origin node id: ' + str(nodes[edges[0].source].id) + '\nDestination node name: ' + str(nodes[edges[0].destination].name) + ', dest origin id: ' + str(nodes[edges[0].destination].id))
    return nodes, edges

    # Save csv with created id 
    #edges_list.to_csv('DGL_data/OD_LT_student_aggre_4326_with_IDs.csv', index= False)

In [84]:
d = 2.0
centroids, testi_matkat = get_locations_data(d)

Total number of edges removed: 0
Nodes removed: 277
dict_values([<__main__.Node object at 0x00000230042CDF70>, <__main__.Node object at 0x00000230042CD850>, <__main__.Node object at 0x000002300454A340>, <__main__.Node object at 0x000002300454AD00>, <__main__.Node object at 0x000002300B13F970>, <__main__.Node object at 0x000002300B13F550>, <__main__.Node object at 0x000002300B13FEE0>, <__main__.Node object at 0x000002300B13FC70>, <__main__.Node object at 0x000002300B13FF10>, <__main__.Node object at 0x000002300B13FE80>, <__main__.Node object at 0x000002300B13FA90>, <__main__.Node object at 0x000002300B18C160>, <__main__.Node object at 0x000002300B18C190>, <__main__.Node object at 0x000002300B18C250>, <__main__.Node object at 0x000002300B18C0D0>, <__main__.Node object at 0x000002300B18C760>, <__main__.Node object at 0x000002300B18C040>, <__main__.Node object at 0x000002300B18C490>, <__main__.Node object at 0x000002300B18C550>, <__main__.Node object at 0x000002300B18C580>, <__main__.Node 

DIJKSTRA.PY

In [133]:
from typing import List
import heapq


def find_shortest_path(source: Node, dest: Node, nodes) -> List[Edge]:
    # reset nodes
    for node in nodes.values():
        node.distance = 99999999999999
        node.visited = False
        node.previous = None
        node.previous_edge = None

    source.distance = 0
    queue = []
    heapq.heappush(queue, (source.distance, source))

    while not len(queue) == 0:
        next_node = heapq.heappop(queue)[1]
        next_node.visited = True

        for edge in next_node.edges:
            if edge.skip:
                continue

            other_id = edge.destination if edge.source == next_node.id else edge.source
            other = nodes[other_id]

            current_distance = next_node.distance + edge.weight

            if current_distance < other.distance:
                other.distance = current_distance
                other.previous = next_node
                other.previous_edge = edge
                heapq.heappush(queue, (other.distance, other))

        # Already found the destination, no need to continue with the search
        if next_node == dest:
            break

    # extract path
    path = []

    node = dest
    while node.previous is not None:
        path.append(node.previous_edge)
        node = node.previous

    path.reverse()
    return path

BEZIER.PY

In [134]:
import numpy as np
import math
from numba import jit


# arguments: list of 2D-np.array, float
# return list
# Check if we have enough control points
@jit(nopython=True)
def eval_bezier(control_points, t):

    if len(control_points) < 2:
        return np.zeros(2)

    if t < 0 or t > 1:
        return np.zeros(2)
    if t == 0:
        return control_points[0]
    if t == 1:
        return control_points[-1]

    # Calculate the intermediate points
    points = control_points
    while len(points) > 1:
        intermediate_points = []
        for i in range(len(points)-1):
            p = (1 - t) * points[i] + t * points[i+1]
            intermediate_points.append(p)
        points = intermediate_points
    return points[0]


# n = number of points to approximate curve
# arguments: list of 2d-np.arrays , int
# returns: list of 2d np.arrays (points) on the bezier curve
@jit(nopython=True)
def create_bezier_polygon(control_points, n):

    if n < 2:
        return [control_points[0], control_points[-1]]
    points = []
    for i in range(n):
        points.append(eval_bezier(control_points, i/n))
    points.append(control_points[-1])
    return points

DRAW.PY

In [135]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

#from bezier import bezierSpherical
from tqdm import tqdm
from shapely.geometry import Point, LineString
#from bezier import bezier as bz


def draw(control_points, nodes, edges, n, use_3d, draw_map, output_option):

    if draw_map:
        nodes_list = centroids
        #map_2 = gpd.read_file('DGL_data/NUTS2_2021_4326/NUTS2_4326.shp')
        geometry = [Point(xy) for xy in zip(nodes_list['X'], nodes_list['Y'])]
        original_routes = gpd.read_file('DGL_data/Real_data/OD_long-term_students_2015.csv', sep=',', encoding="utf-8", usecols=['OD_ID','ORIGIN','DESTINATION','COUNT'])
        
        #nodes_list = create_ids(nodes_list) 

        geo_df = gpd.GeoDataFrame(nodes_list, crs='epsg:4326', geometry=geometry)
        #testaan miltä näyttää koordinaatit orig/dest-sovitusta varten 1
        print('\nNODE_LIST ALKU \n',geo_df.head())
        

        #fig, ax = plt.subplots(figsize=(50, 25))
        #map.plot(ax=ax, alpha=0.4, color='grey')
        #geo_df.plot(ax=ax, markersize=1)
    else:
        plt.gcf().set_dpi(300)

    if use_3d: #meillä ei käyttöä
        n = -1
        step_size = 5
        #bezierSpherical.plot_spherical(control_points, nodes, edges, n, step_size)
    else:
        # create and bezier curves
        bezier_polygons = []
        for controlPoints in tqdm(control_points, desc="Drawing curves: "):
            polygon = create_bezier_polygon(controlPoints, n)  # returns list of 2d vectors
            bezier_polygons.append(polygon)
            x = [arr[0] for arr in polygon]
            y = [arr[1] for arr in polygon]
            #plt.plot(x, y, color='red', linewidth=0.1, alpha=1)
        print('\nPRINTING 1. BEZIER \n', bezier_polygons[0])
        
        # create a list for the first and last points of each bezier line
        cp_list = [] #lista 1. ja vika control_points
        for poly in bezier_polygons:
            cp_list.append(np.array([poly[0], poly[-1]]))
                           
        print('\nCONTROL POINTS LISTAN 1. \n', cp_list[0])
        
        # create a dataframe from control points to create origin and destination coordinate points
        cp_df = pd.DataFrame(columns=['orig', 'dest'])
        for cp in cp_list:
            cp_df = cp_df.append({'orig': cp[0], 'dest': cp[1]}, ignore_index=True)
        # adding an index column to the OD dataframe
        cp_df['id'] = cp_df.index
        
        print('\nBEZIER OD_DF HEAD \n', cp_df.head())
        
        # adding orig_nuts and dest_nuts ID's for control_points for later on happening join
        for index, poly in cp_df.iterrows():
            for _, centroid in geo_df.iterrows():
                if (poly['orig'][0] == centroid['X']) and (poly['orig'][1] == centroid['Y']):
                    cp_df.at[index, 'orig_nuts'] = centroid['NUTS_ID']

        for index, poly in cp_df.iterrows():
            for _, centroid in geo_df.iterrows():
                if (poly['dest'][0] == centroid['X']) and (poly['dest'][1] == centroid['Y']):
                    cp_df.at[index, 'dest_nuts'] = centroid['NUTS_ID']
                
        print('\nBEZIER OD_DF HEAD AFTER NUTS_ID JOIN \n', cp_df.head())
        
        # create list of bezier linestrings
        lines = []
        for poly in bezier_polygons:
            a = LineString(poly)
            lines.append(a)

        # create dataframe and geodataframe for bezier lines
        lines_df = pd.DataFrame(lines, columns=['geometry'])
        
        lines_gdf = gpd.GeoDataFrame(lines_df, crs='epsg:4326', geometry='geometry')
        # adding an index column to the lines dataframe
        lines_gdf['id'] = lines_gdf.index
        
        # merging orig_nuts and dest_nuts to lines_gdf
        merged_lines_gdf = lines_gdf.merge(cp_df[['id','orig_nuts','dest_nuts']], on='id', how='left')
        
        # generate od ids
        merged_lines_gdf['OD_ID'] = merged_lines_gdf['orig_nuts'] + '_' + merged_lines_gdf['dest_nuts']
        
        # join original count data based on od_id
        merged_counts = merged_lines_gdf.merge(original_routes[['OD_ID','COUNT']], on=['OD_ID'], how='left')
        
        merged_counts['COUNT'] = merged_counts['COUNT'].astype(int)

        #testaan miltä näyttää koordinaatit orig/dest-sovitusta varten 2
        print('\nPRINTING lines_gdf HEAD \n',merged_counts.head())
        
        # empty list for dataframes
        straight_edges = []

        # draw lines without detour or with detour that was too long
        for edge in tqdm(edges, desc="Drawing lines: "):
            if edge.skip:
                continue
            
            # get nodes
            o = nodes[edge.source]
            d = nodes[edge.destination]
            
            # get names of nodes
            o_name = o.name
            d_name = d.name
                        
            # get count of flow for edge connecting the nodes
            count = edge.count

            # generate geometry for the edge
            line = LineString([Point([o.longitude, o.latitude]), Point([d.longitude, d.latitude])])
            
            # old lat lon stuff
            x = [o.longitude, d.longitude]
            y = [o.latitude, d.latitude]
            
            # create dataframe
            straight_df = pd.DataFrame(columns=['orig_nuts', 'dest_nuts', 'OD_ID', 'COUNT', 'geometry'])
            
            # add row
            straight_df = straight_df.append({'orig_nuts': o_name,
                                              'dest_nuts': d_name,
                                              'OD_ID': o_name + '_' + d_name,
                                              'COUNT': count,
                                              'geometry': line}, ignore_index=True)
                    
            # add to list
            straight_edges.append(straight_df)
            
            #plt.plot(x, y, color='red', linewidth=0.1, alpha=1)
        
      
        # concatenate list to dataframe
        straights = pd.concat(straight_edges)
        
        # add orig and dest nuts
        straights['orig_nuts'] = straights['OD_ID'].apply(lambda x: x.split('_')[0])
        straights['dest_nuts'] = straights['OD_ID'].apply(lambda x: x.split('_')[1])
        
        # create geodataframe of straight lines
        straights = gpd.GeoDataFrame(straights, crs='epsg:4326', geometry='geometry')
        
        #testaan miltä näyttää koordinaatit orig/dest-sovitusta varten 2
        print('\nPRINTING straights head \n',straights.head(n=2))
        
        # get results
        results = merged_counts.append(straights, ignore_index=True)
        
        # choose file output options
        if output_option == 1:
            results.to_file('Test_lines/Filter_test_FI_LT.shp')
            
        elif output_option == 2:
            straights.to_file('Real_results_lines/2015_straight_lines_LT.shp')
            merged_counts.to_file('Real_results_lines/2015_Bezier_LT.shp')
            
        elif output_option == 3:
            results.to_file('Real_results_lines/2015_straights_AND_bezier_LT.shp')
            straights.to_file('Real_results_lines/2015_straight_lines_LT.shp')
            merged_counts.to_file('Real_results_lines/2015_Bezier_LT.shp')
        
            
        

    #plt.axis('scaled')
    #plt.axis('off')
    #plt.tight_layout()
    #plt.show()

MAIN.PY

In [144]:
from tqdm import tqdm
#from bezier import control_points
#from drawing import draw

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

############################################################################################
# PARAMETERS
############################################################################################

d = 2.0  # edge weight parameter, weight = distance^d
k = 2.0  # if new detour is longer than k times the original line it is not bundled
n = 100  # number of sampling points in Bezier curves
draw_map = True  # Draw map for the underlying data
plot_3d = False  # Plot for use on sphere
smoothing = 2  # Smoothing parameter for Bezier curves
output_option = 1 # Choose drawing output file option

# data source
nodes, edges = get_locations_data(d)
# nodes, edges = migrations.get_migrations_data(d)

############################################################################################
# MAIN CYCLE
############################################################################################

control_point_lists = []
too_long = 0
no_path = 0
for edge in tqdm(edges, desc="Computing: "):
    if edge.lock:
        continue

    edge.skip = True

    source = nodes[edge.source]
    dest = nodes[edge.destination]

    path = find_shortest_path(source, dest, nodes)

    if len(path) == 0:
        no_path += 1
        edge.skip = False
        continue

    original_edge_distance = source.distance_to(dest)
    new_path_length = sum([e.distance for e in path])

    if new_path_length > k * original_edge_distance:
        too_long += 1
        edge.skip = False
        continue

    for edge_in_path in path:
        edge_in_path.lock = True

    # Get control points for drawing
    control_point_lists.append(get(source, dest, nodes, path, smoothing))
print('Control point lists',control_point_lists[:1])

############################################################################################
# DRAWING
############################################################################################
draw(control_point_lists, nodes, edges, n, plot_3d, draw_map, output_option)

print(f"Out of {len(edges)} edges, {too_long} had too long detour and {no_path} had no alternative path.")


Total number of edges removed: 0
Nodes removed: 296
dict_values([<__main__.Node object at 0x000002300FBE3730>, <__main__.Node object at 0x000002300FBE3700>, <__main__.Node object at 0x000002300FBE3760>, <__main__.Node object at 0x000002300FBE37C0>, <__main__.Node object at 0x000002300FBE3880>, <__main__.Node object at 0x000002300FBE38B0>, <__main__.Node object at 0x000002300FBE38E0>, <__main__.Node object at 0x000002300FBE3A00>, <__main__.Node object at 0x000002300FBE3A60>, <__main__.Node object at 0x000002300FBE3DF0>, <__main__.Node object at 0x000002300FBE3AC0>, <__main__.Node object at 0x000002300FBE3BB0>, <__main__.Node object at 0x000002300FBE3FD0>, <__main__.Node object at 0x000002300FBFC070>, <__main__.Node object at 0x000002300FBFC490>, <__main__.Node object at 0x000002300FBFC9A0>, <__main__.Node object at 0x000002300FBFC9D0>, <__main__.Node object at 0x000002300FBFCB50>, <__main__.Node object at 0x000002300FBFCBB0>, <__main__.Node object at 0x000002300FBFCC40>, <__main__.Node 

Computing: 100%|█████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 6476.47it/s]

Control point lists []


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 80: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
Traceback (most recent call last):
  File "C:\Users\oulain\miniconda3\envs\python-gis\lib\site-packages\fiona\env.py", line 283, in defenv
    local._env.start()
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 80: invalid continuation byte


...

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 75: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
Traceback (most recent call last):
  File "C:\Users\oulain\miniconda3\envs\python-gis\lib\site-packages\fiona\env.py", line 283, in defenv
    local._env.start()
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 75: invalid continuation byte



NODE_LIST ALKU 
   NUTS_ID  LEVL_CODE CNTR_CODE         NAME_LATN         NUTS_NAME  \
0    CZ05          2        CZ     SeverovÃ½chod     SeverovÃ½chod   
1    CZ06          2        CZ       JihovÃ½chod       JihovÃ½chod   
2    CZ07          2        CZ  StÅednÃ­ Morava  StÅednÃ­ Morava   
3    CZ08          2        CZ   Moravskoslezsko   Moravskoslezsko   
4    DE11          2        DE         Stuttgart         Stuttgart   

   MOUNT_TYPE  URBN_TYPE  COAST_TYPE   FID       X       Y  \
0         0.0          0           0  CZ05  15.773  50.283   
1         0.0          0           0  CZ06  16.171  49.241   
2         0.0          0           0  CZ07  17.430  49.539   
3         0.0          0           0  CZ08  17.986  49.822   
4         0.0          0           0  DE11   9.617  49.026   

                    geometry  
0  POINT (15.77300 50.28300)  
1  POINT (16.17100 49.24100)  
2  POINT (17.43000 49.53900)  
3  POINT (17.98600 49.82200)  
4   POINT (9.61700 49.02600)  


Drawing curves: : 0it [00:00, ?it/s]


IndexError: list index out of range